# Notebook for simulating, saving and analysing MOMA-like data with "fuzzy" initital parameters


In [1]:
from cells_simulation import *

In [2]:
# ========== Simulation parameters ========== #
save_dataset = True    # saves data set as csv
run_ggp = True         # runs the ggp code (located in "../bin/" relative to this nb)

dt = 1e-3
dt_measument = 3 # in minutes
n_cells = 250 # number of cells that will be simulated

div_mode = "adder"
division_log_length = 1+np.log(2)   # for sizer: division, when log_length hits division_log_length
division_time = 60 - 1e-10          # for timer: division, when cell cycle time hits division_time
division_addition = np.log(2)       # for adder: divsion, when division_addition in log_length was added in cell cycle

# ========== Model parameters ========== #
parameter_set = {   "mean_lambda": 0.01, 
                    "gamma_lambda": 0.01,
                    "var_lambda": 1e-7,
                    "mean_q": 10,
                    "gamma_q": 1e-2,
                    "var_q": 0.1,
                    "beta": 0.001, #1e-3,
                    "var_x": 1e-3,
                    "var_g": 5000,
                    "var_dx": 0.001,
                    "var_dg": 500}

# --------- SIMULATE --------- #
cells_simulated = simulate_cells(dt, n_cells, parameter_set, div_mode,
                         division_log_length, 
                         division_time, 
                         division_addition)

dataset = build_data_set(cells_simulated, parameter_set['var_x'], parameter_set['var_g'], int(dt_measument/dt))


| ==================== | Progress 100%  No of cells:  250
Every 3000 th datapoint is saved


In [5]:
def write_param_file(filename, parameters, non_default={}):
    with open(filename, "w") as fin:
        fin.write("# Generated config file for simulated data\n")
        for k, v in parameters.items():
            if k in non_default:
                if non_default[k][0] == "fixed":
                    fin.write("{:s} = {:.2E}\n".format(k, v))
                elif non_default[k][0] == "free":
                    fin.write("{:s} = {:.2E}, {:.2E}\n".format(k, v, v))
                else:
                    fin.write("{:s} = {:.2E}, {:.2E}, {:.2E}, {:.2E}\n".format(k,   v*non_default[k][0],    
                                                                                    v*non_default[k][1], 
                                                                                    v*non_default[k][2], 
                                                                                    v*non_default[k][3] ))
            else:
                if v==0:
                    fin.write("{:s} = {:.2E}, {:.2E}\n".format(k, v, 1e-3)) 
                else:
                    fin.write("{:s} = {:.2E}, {:.2E}, {:.2E}, {:.2E}\n".format(k, v, v*1e-2, 0, v*100 ))

In [8]:
def add_gaussian_noise2params(parameters, rel_noise=0.1, rel_min=0.01):
    new = copy.deepcopy(parameters)
    for k, v in new.items():
        temp = new[k] + np.random.normal(loc=0, scale=new[k]*rel_noise)
        new[k] = np.max([temp, v*rel_min])
    return new

non_default = dict()

# col = []
# for i in range(5000):
#     col.append(add_uni_noise2params(parameter_set)["var_g"])

# plt.hist(col, 100)

In [10]:
out_dir = mk_mising_dir('../data/fuzzy_param_guess_adder')

noise_levels = [10 for i in range(10)]
print(*noise_levels, "\n")

for param_noise in noise_levels:
    # ----------- SAVE ----------- #
    if save_dataset:
        directory, filename = get_next_file_name(out_dir)

        fuzzy_params = add_gaussian_noise2params(parameter_set, param_noise)
        write_param_file(os.path.join(directory, "parameters.txt"), fuzzy_params, non_default) 

        write_csv_config(os.path.join(directory, "csv_config.txt"))
        dataset.to_csv(filename)
        print("New simulation saved in", filename)

        # ----------- RUN ----------- #
        if run_ggp:
            run_cmd = suggest_run_command(directory, filename, modes="-m -p")
            print("run: -> \n", run_cmd)
            s = subprocess.run(run_cmd, capture_output=False, shell=True)
            print(s)
        

10 10 10 10 10 10 10 10 10 10 

New simulation saved in ../data/fuzzy_param_guess_adder/fuzzy_param_guess_adder_0/fuzzy_param_guess_adder_0.csv
run: -> 
 ../bin/gfp_gaussian -c ../data/fuzzy_param_guess_adder/fuzzy_param_guess_adder_0/csv_config.txt -b ../data/fuzzy_param_guess_adder/fuzzy_param_guess_adder_0/parameters.txt -r 1e-3  -i ../data/fuzzy_param_guess_adder/fuzzy_param_guess_adder_0/fuzzy_param_guess_adder_0.csv -l 0 -m -p


KeyboardInterrupt: 